In [29]:
import pandas as pd
import folium
import chardet
from IPython.display import display

# 감지된 인코딩으로 CSV 파일 로드
df = pd.read_csv('/content/sample (1).csv', encoding='euc-kr')

# 시간대와 요일, 빨간색 표시할 좌표를 받아 지도를 생성하는 함수
def plot_coordinates_for_day(df, day_of_week, start_time, end_time, red_marker_coords=None):
    # 시간 필터링을 위해 승차시간을 datetime 형식으로 변환
    df['승차시간'] = pd.to_datetime(df['승차시간'])

    # 주어진 시간대에 해당하는 데이터 필터링
    mask = (df['승차시간'].dt.time >= pd.to_datetime(start_time).time()) & \
           (df['승차시간'].dt.time <= pd.to_datetime(end_time).time()) & \
           (df['요일'] == day_of_week)
    filtered_df = df[mask]

    # 지도 생성 (중심 좌표는 데이터의 중앙으로 설정)
    center_lat = df['승차Y좌표'].mean()
    center_lon = df['승차X좌표'].mean()
    mymap = folium.Map(location=[center_lat, center_lon], zoom_start=13)

    # 필터링된 데이터의 각 좌표에 대해 지도에 마커 추가
    for idx, row in filtered_df.iterrows():
        folium.Marker([row['승차Y좌표'], row['승차X좌표']]).add_to(mymap)

    # 빨간색으로 표시할 특정 좌표 추가 (선택적, X좌표, Y좌표 순으로 입력)
    if red_marker_coords:
        folium.Marker(location=[red_marker_coords[1], red_marker_coords[0]], icon=folium.Icon(color='red')).add_to(mymap)

    # 지도 출력
    display(mymap)

# 예시: Saturday의 00:00~03:00 시간대의 좌표 출력, 특정 좌표(127.38806932302126, 36.34675257595036)를 빨간색으로 표시
plot_coordinates_for_day(df, 'Saturday', '02:30:20', '02:40:40', red_marker_coords=[127.38806932302126, 36.34675257595036])


In [27]:
import pandas as pd
from sklearn.cluster import DBSCAN
from datetime import datetime, timedelta
import numpy as np

# CSV 파일 로드
df = pd.read_csv('/content/sample (1).csv', encoding='euc-kr')

# 차량번호 데이터 제거
df = df.drop(columns=['차량번호'])

# 승차시간을 datetime으로 변환
df['승차시간'] = pd.to_datetime(df['승차시간'])

# 15분 단위로 시간 그룹화
df['시간그룹'] = df['승차시간'].dt.floor('15T')

# DBSCAN 클러스터링을 위한 데이터 준비
# 클러스터링에는 X좌표, Y좌표가 필요합니다
coords = df[['승차X좌표', '승차Y좌표']].values

# DBSCAN 적용
db = DBSCAN(eps=0.01, min_samples=5).fit(coords)  # eps와 min_samples는 데이터에 따라 조정 가능
df['클러스터'] = db.labels_

# 클러스터 중심점 계산
centroids = df.groupby(['클러스터'])[['승차X좌표', '승차Y좌표']].mean()

# 클러스터 중심점과 시간그룹, 요일을 연결
df = df.merge(centroids, how='left', on='클러스터', suffixes=('', '_중심'))

# 요일과 시간에 따른 추천 함수 정의
def recommend_location(day_of_week, time):
    # 입력된 시간과 요일에 맞는 데이터를 필터링
    time_group = pd.to_datetime(time).floor('15T')
    filtered_df = df[(df['요일'] == day_of_week) & (df['시간그룹'] == time_group)]

    if not filtered_df.empty:
        # 가장 빈도가 높은 클러스터의 중심 좌표를 반환
        recommended_cluster = filtered_df['클러스터'].mode()[0]
        location = centroids.loc[recommended_cluster]
        return location['승차X좌표'], location['승차Y좌표']
    else:
        return None, None

# 사용 예시
day = 'Saturday'
time = '2023-04-01 02:30:00'
x, y = recommend_location(day, time)
print(f"추천 위치: X = {x}, Y = {y}")

추천 위치: X = 127.38806932302126, Y = 36.34675257595036
